In [1]:
from datetime import datetime

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import patsy

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

from scipy.stats import ks_2samp

In [2]:
#  Abrindo o dataset
df = pd.read_excel(r"C:\Users\sabri\EuropeTeams.xlsx")
df.head()

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals,assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded
0,00474e7e-f4e1-4be5-9a09-f3993737d2ff,EU U TL vs VIT G1 2023-02-24.18.04,Mannfield (Night),2023-02-24,loss,TEAM LIQUID,TEAM VITALITY,816,1,1,4,6,8,16.666667,5,1,0.500000
1,00474e7e-f4e1-4be5-9a09-f3993737d2ff,EU U TL vs VIT G1 2023-02-24.18.04,Mannfield (Night),2023-02-24,win,TEAM VITALITY,TEAM LIQUID,1172,3,2,4,8,6,37.500000,1,5,0.666667
2,00a061a7-1ff0-4d91-8e50-b66b6b15540b,EU A KC vs SLY G4 2023-02-24.16.29,Utopia Coliseum (Dusk),2023-02-24,loss,KARMINE CORP,SOLARY,570,0,0,3,5,7,0.000000,2,2,0.428571
3,00a061a7-1ff0-4d91-8e50-b66b6b15540b,EU A KC vs SLY G4 2023-02-24.16.29,Utopia Coliseum (Dusk),2023-02-24,win,SOLARY,KARMINE CORP,931,1,1,4,7,5,14.285714,2,2,0.800000
4,00adb747-d918-4818-b41c-37917eb18e19,EU P TL vs VIT G5 2023-05-14.19.56,NaN,2023-05-14,win,TEAM LIQUID,TEAM VITALITY,1750,3,3,9,19,11,15.789474,1,1,0.818182


In [3]:
#  Renomear a coluna assists para goals_assists e goals para goals_solo
df.rename(columns={'assists':'goals_assists'}, inplace = True)
df.rename(columns={'goals':'goals_solo'}, inplace = True)
df

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded
0,00474e7e-f4e1-4be5-9a09-f3993737d2ff,EU U TL vs VIT G1 2023-02-24.18.04,Mannfield (Night),2023-02-24,loss,TEAM LIQUID,TEAM VITALITY,816,1,1,4,6,8,16.666667,5,1,0.500000
1,00474e7e-f4e1-4be5-9a09-f3993737d2ff,EU U TL vs VIT G1 2023-02-24.18.04,Mannfield (Night),2023-02-24,win,TEAM VITALITY,TEAM LIQUID,1172,3,2,4,8,6,37.500000,1,5,0.666667
2,00a061a7-1ff0-4d91-8e50-b66b6b15540b,EU A KC vs SLY G4 2023-02-24.16.29,Utopia Coliseum (Dusk),2023-02-24,loss,KARMINE CORP,SOLARY,570,0,0,3,5,7,0.000000,2,2,0.428571
3,00a061a7-1ff0-4d91-8e50-b66b6b15540b,EU A KC vs SLY G4 2023-02-24.16.29,Utopia Coliseum (Dusk),2023-02-24,win,SOLARY,KARMINE CORP,931,1,1,4,7,5,14.285714,2,2,0.800000
4,00adb747-d918-4818-b41c-37917eb18e19,EU P TL vs VIT G5 2023-05-14.19.56,NaN,2023-05-14,win,TEAM LIQUID,TEAM VITALITY,1750,3,3,9,19,11,15.789474,1,1,0.818182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2755,ff8636e4-18b7-445f-a0e2-127865f10bbf,EU KC vs G1 G1 2022-10-28.18.07,Mannfield (Night),2022-10-28,loss,KARMINE CORP,G1,1771,2,2,8,19,12,10.526316,6,6,0.666667
2756,ffb408ac-8c69-473e-ac67-73d0b42e84f9,EU I TL vs QUAD G4 2022-10-28.17.22,Utopia Coliseum (Dusk),2022-10-28,win,QUADRANT,TEAM LIQUID,1536,3,2,10,10,14,30.000000,1,4,0.714286
2757,ffb408ac-8c69-473e-ac67-73d0b42e84f9,EU I TL vs QUAD G4 2022-10-28.17.22,Utopia Coliseum (Dusk),2022-10-28,loss,TEAM LIQUID,QUADRANT,1096,1,1,6,14,10,7.142857,4,1,0.600000
2758,fffbcd16-ac1c-49ac-8d81-ab94021d4165,EU V WRSV vs FUF G5,Champions Field,2023-02-24,win,FUFAXDOP,WILLIAMS RESOLVE,1214,3,3,3,10,7,30.000000,4,6,0.428571


In [4]:
#  Consertando os valores da variável goals solo para excluir os valores de goals_assists
df['goals_solo'] = df['goals_solo'] - df['goals_assists']
df

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded
0,00474e7e-f4e1-4be5-9a09-f3993737d2ff,EU U TL vs VIT G1 2023-02-24.18.04,Mannfield (Night),2023-02-24,loss,TEAM LIQUID,TEAM VITALITY,816,0,1,4,6,8,16.666667,5,1,0.500000
1,00474e7e-f4e1-4be5-9a09-f3993737d2ff,EU U TL vs VIT G1 2023-02-24.18.04,Mannfield (Night),2023-02-24,win,TEAM VITALITY,TEAM LIQUID,1172,1,2,4,8,6,37.500000,1,5,0.666667
2,00a061a7-1ff0-4d91-8e50-b66b6b15540b,EU A KC vs SLY G4 2023-02-24.16.29,Utopia Coliseum (Dusk),2023-02-24,loss,KARMINE CORP,SOLARY,570,0,0,3,5,7,0.000000,2,2,0.428571
3,00a061a7-1ff0-4d91-8e50-b66b6b15540b,EU A KC vs SLY G4 2023-02-24.16.29,Utopia Coliseum (Dusk),2023-02-24,win,SOLARY,KARMINE CORP,931,0,1,4,7,5,14.285714,2,2,0.800000
4,00adb747-d918-4818-b41c-37917eb18e19,EU P TL vs VIT G5 2023-05-14.19.56,NaN,2023-05-14,win,TEAM LIQUID,TEAM VITALITY,1750,0,3,9,19,11,15.789474,1,1,0.818182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2755,ff8636e4-18b7-445f-a0e2-127865f10bbf,EU KC vs G1 G1 2022-10-28.18.07,Mannfield (Night),2022-10-28,loss,KARMINE CORP,G1,1771,0,2,8,19,12,10.526316,6,6,0.666667
2756,ffb408ac-8c69-473e-ac67-73d0b42e84f9,EU I TL vs QUAD G4 2022-10-28.17.22,Utopia Coliseum (Dusk),2022-10-28,win,QUADRANT,TEAM LIQUID,1536,1,2,10,10,14,30.000000,1,4,0.714286
2757,ffb408ac-8c69-473e-ac67-73d0b42e84f9,EU I TL vs QUAD G4 2022-10-28.17.22,Utopia Coliseum (Dusk),2022-10-28,loss,TEAM LIQUID,QUADRANT,1096,0,1,6,14,10,7.142857,4,1,0.600000
2758,fffbcd16-ac1c-49ac-8d81-ab94021d4165,EU V WRSV vs FUF G5,Champions Field,2023-02-24,win,FUFAXDOP,WILLIAMS RESOLVE,1214,0,3,3,10,7,30.000000,4,6,0.428571


In [5]:
#  Checando se há empates nos resultados
df_draws = df[df['result']=='draw']
df_draws

,replay_id,replay_title,map,date,result,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded


In [6]:
#  Transformando result em variável binária. Loss = 1 e win = 0
df1 = pd.get_dummies(df['result'])
#df1.head()
df1 = df1.drop(['win'], axis = 1)
#df1.head()
df = pd.concat((df, df1), axis = 1)
#df.head()
df = df.drop(['result'], axis = 1)
df = df.rename(columns = {'loss': 'result'})
df.head()

,replay_id,replay_title,map,date,team_name,opposing_team_name,score,goals_solo,goals_assists,saves,shots,shots_conceded,shooting_percentage,demos_inflicted,demos_taken,save_conceded,result
0,00474e7e-f4e1-4be5-9a09-f3993737d2ff,EU U TL vs VIT G1 2023-02-24.18.04,Mannfield (Night),2023-02-24,TEAM LIQUID,TEAM VITALITY,816,0,1,4,6,8,16.666667,5,1,0.500000,1
1,00474e7e-f4e1-4be5-9a09-f3993737d2ff,EU U TL vs VIT G1 2023-02-24.18.04,Mannfield (Night),2023-02-24,TEAM VITALITY,TEAM LIQUID,1172,1,2,4,8,6,37.500000,1,5,0.666667,0
2,00a061a7-1ff0-4d91-8e50-b66b6b15540b,EU A KC vs SLY G4 2023-02-24.16.29,Utopia Coliseum (Dusk),2023-02-24,KARMINE CORP,SOLARY,570,0,0,3,5,7,0.000000,2,2,0.428571,1
3,00a061a7-1ff0-4d91-8e50-b66b6b15540b,EU A KC vs SLY G4 2023-02-24.16.29,Utopia Coliseum (Dusk),2023-02-24,SOLARY,KARMINE CORP,931,0,1,4,7,5,14.285714,2,2,0.800000,0
4,00adb747-d918-4818-b41c-37917eb18e19,EU P TL vs VIT G5 2023-05-14.19.56,NaN,2023-05-14,TEAM LIQUID,TEAM VITALITY,1750,0,3,9,19,11,15.789474,1,1,0.818182,0


In [7]:
#  Excluindo várias colunas de uma vez
df.drop(['replay_id','replay_title','map','team_name','opposing_team_name', 'shots_conceded'], axis = 1, inplace = True)

In [8]:
#  Apagando as observações vazias
df.dropna(axis = 0, inplace = True)

In [9]:
#  Dividindo a base de validação
periodo_des = df[df['date'] < datetime(2023,6,1)].copy()
oot = df[df['date'] > datetime(2023,6,1)].copy()
periodo_des.reset_index(inplace = True)
oot.reset_index(inplace=True)

print(periodo_des.shape, oot.shape)

(2496, 12) (264, 12)


In [10]:
#  Definindo x e y
x_oot = oot.drop(['result', 'date', 'index'], axis = 1)
y_oot = oot['result']
x = periodo_des.drop(['result', 'date', 'index'], axis = 1)
y = periodo_des['result']
x_oot.shape

(264, 9)

In [11]:
#  Separando a base de desenvolvimento em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)
x_train.shape, x_test.shape

((1872, 9), (624, 9))

In [12]:
#  Definindo o modelo de trabalho - Gradient Boosting

model = GradientBoostingClassifier()
model.fit(x_train, y_train)

#  Predict
# predict the mode
y_pred = model.predict(x_test)
classification_report(y_test, y_pred)
# performance evaluation metrics
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.88      0.90      0.89       309
           1       0.90      0.88      0.89       315

    accuracy                           0.89       624
   macro avg       0.89      0.89      0.89       624
weighted avg       0.89      0.89      0.89       624



In [13]:
#  Definindo parâmetros
param_grid = {'n_estimators': [10, 100, 300, 600] 
              , 'min_samples_leaf': [1, 10, 20, 50]
              , 'learning_rate': [.05, .1, .25, .5, 1]
             }

In [14]:
#  Separando os valores de cv
#  **parâmetro CV:** Significa *cross validation* (ou validação cruzada). 
#  Nele indicamos os índices das observações que serão utilizadas para desenvolver o modelo e para validar respectivamente. 
#  Através deste parâmetro, a função ainda consegue fazer outras modalidades de validação cruzada.

index_des = periodo_des[periodo_des.index%2 != 0]
index_des1 = periodo_des[periodo_des.index%2 == 0]
indice_des = index_des.index.values
indice_oos = index_des1.index.values
print(indice_des)
print(indice_oos)

[   1    3    5 ... 2491 2493 2495]
[   0    2    4 ... 2490 2492 2494]


In [15]:
#  Fazendo a procura dos melhores parâmetros no modelo
grid_search = GridSearchCV(model,
                           param_grid=param_grid,
                          scoring = 'roc_auc',
                          cv = [[indice_des, indice_oos]])

grid_search.fit(x, y)
print(grid_search.best_estimator_)

GradientBoostingClassifier(learning_rate=0.25, min_samples_leaf=20)


In [20]:
#  Aplicar melhores resultados aqui
model = GradientBoostingClassifier(min_samples_leaf = 20, learning_rate = 0.25)
model.fit(x_train, y_train)


# predict the mode
y_pred = model.predict(x_test)
av1 = classification_report(y_test, y_pred)
y_pred1 = model.predict(x_oot)
av2 = classification_report(y_oot, y_pred1)
# performance evaluatio metrics
print(av1, av2)

              precision    recall  f1-score   support

           0       0.88      0.87      0.88       316
           1       0.87      0.88      0.88       308

    accuracy                           0.88       624
   macro avg       0.88      0.88      0.88       624
weighted avg       0.88      0.88      0.88       624
               precision    recall  f1-score   support

           0       0.88      0.86      0.87       132
           1       0.87      0.88      0.87       132

    accuracy                           0.87       264
   macro avg       0.87      0.87      0.87       264
weighted avg       0.87      0.87      0.87       264



In [21]:
#  Avaliando modelo na base de treino
y_pred0 = model.predict(x_train)
av0 = classification_report(y_train, y_pred0)
print(av0)

              precision    recall  f1-score   support

           0       0.95      0.95      0.95       932
           1       0.95      0.95      0.95       940

    accuracy                           0.95      1872
   macro avg       0.95      0.95      0.95      1872
weighted avg       0.95      0.95      0.95      1872



In [23]:
#  Separando as features para identificação da importância das variáveis
feature_list = list(x.columns)
#  Importância das variáveis
importances = list(model.feature_importances_)
# Print out the feature and importances 
print(feature_list, importances)

['score', 'goals_solo', 'goals_assists', 'saves', 'shots', 'shooting_percentage', 'demos_inflicted', 'demos_taken', 'save_conceded'] [0.08302088999222552, 0.10218881153180633, 0.21898165528139174, 0.07833197896108941, 0.03766512773901987, 0.24184386173756295, 0.004970443745364466, 0.0052312728411427, 0.22776595817039708]
